In [1]:
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?"],
              "responses": ["Salut à toi!", "Hello", "Comment vas tu?", "Salutations!", "Enchanté"],
             },
             {"tag": "age",
              "patterns": ["Quel âge as-tu?", "C'est quand ton anniversaire?", "Quand es-tu né?"],
              "responses": ["J'ai 25 ans", "Je suis né en 1996", "Ma date d'anniversaire est le 3 juillet et je suis né en 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["Que fais-tu ce week-end?",
"Tu veux qu'on fasse un truc ensemble?", "Quels sont tes plans pour cette semaine"],
              "responses": ["Je suis libre toute la semaine", "Je n'ai rien de prévu", "Je ne suis pas occupé"]
             },
             {"tag": "name",
              "patterns": ["Quel est ton prénom?", "Comment tu t'appelles?", "Qui es-tu?"],
              "responses": ["Mon prénom est Miki", "Je suis Miki", "Miki"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "Salut", "see ya", "adios", "cya"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!"]
             }
]}

In [2]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
doc_X = []
doc_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

words = sorted(set(words))
classes = sorted(set(classes))

In [4]:
print(words)

['adios', 'anniversaire', 'as-tu', 'bye', "c'est", 'ce', 'cette', 'comment', 'cya', 'ensemble', 'es-tu', 'est', 'fais-tu', 'fasse', 'forme', 'hello', 'la', 'né', 'plan', 'pour', 'prénom', "qu'on", 'quand', 'que', 'quel', 'quels', 'qui', 'roule', 'salut', 'see', 'semaine', 'sont', "t'appelles", 'te', 'ton', 'truc', 'tu', 'un', 'veux', 'week-end', 'ya', 'yo', 'âge', 'ça']


In [5]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [6]:
print(doc_X)

['Hello', 'La forme?', 'yo', 'Salut', 'ça roule?', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'Quel est ton prénom?', "Comment tu t'appelles?", 'Qui es-tu?', 'bye', 'Salut', 'see ya', 'adios', 'cya']


In [7]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [8]:
# Training data list
training = []
out_empty = [0] * len(classes)
# Word ensemble model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # OHE 
    training.append([bow, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
# Separate features and target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [9]:
# Model parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

In [10]:
# DL model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5760      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 14,341
Trainable params: 14,341
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# Training
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 543ms/step - loss: 1.6313 - accuracy: 0.2632
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 1.5417 - accuracy: 0.2632
Epoch 3/200
1/1 [==============================] - 0s 5ms/step - loss: 1.4779 - accuracy: 0.4737
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 1.3061 - accuracy: 0.6316
Epoch 5/200
1/1 [==============================] - 0s 7ms/step - loss: 1.2282 - accuracy: 0.6842
Epoch 6/200
1/1 [==============================] - 0s 5ms/step - loss: 1.1633 - accuracy: 0.6842
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.9686 - accuracy: 0.8421
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 1.1341 - accuracy: 0.8421
Epoch 9/200
1/1 [==============================] - 0s 4ms/step - loss: 0.8823 - accuracy: 0.7895
Epoch 10/200
1/1 [==============================] - 0s 3ms/step - loss: 0.7188 - accuracy: 0.8947
Epoch 11/200
1/1 [=========

In [13]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)
def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [14]:
# Chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

1/1 [==============================] - 0s 73ms/step
C'était sympa de te parler
1/1 [==============================] - 0s 17ms/step
C'était sympa de te parler
1/1 [==============================] - 0s 15ms/step
à plus tard
1/1 [==============================] - 0s 15ms/step
C'était sympa de te parler
1/1 [==============================] - 0s 14ms/step
à plus tard
1/1 [==============================] - 0s 15ms/step
à plus tard
1/1 [==============================] - 0s 14ms/step
On se reparle très vite!
1/1 [==============================] - 0s 14ms/step
à plus tard
1/1 [==============================] - 0s 16ms/step
On se reparle très vite!
1/1 [==============================] - 0s 15ms/step
Je suis libre toute la semaine
1/1 [==============================] - 0s 15ms/step
J'ai 25 ans
1/1 [==============================] - 0s 13ms/step
Je suis né en 1996
1/1 [==============================] - 0s 17ms/step
à plus tard
1/1 [==============================] - 0s 14ms/step
J'ai 25 ans
1/1 [==